In [6]:
# import library
import pandas as pd
import numpy as np 
import matlab 
import matplotlib as plt
import os
import numpy as np


In [7]:
# read the file
# mortality
mortality_path = r'data/Morticd10_part1.csv'
mortality_data_2002 = pd.read_csv(mortality_path, low_memory=False)

In [8]:
# population
pop_path = r'data/pop.csv'
pop_data = pd.read_csv(pop_path, low_memory=False)

In [9]:
# country code
country_code_path = r'data/country_codes.csv'
country_code = pd.read_csv(country_code_path, low_memory=False)

In [10]:
# changes to the mortality data file, creating a new data frame with only the number of deaths and the population, country, year, deaths1 - deaths26, pop1 - pop 26, im_death
# creating the country code map 
country_code_map = {}
for index, rows in country_code.iterrows():
    country_code_map[rows['country']] = rows['name']
    

In [13]:
# looping through the code map to locate each different countries
# condense the mortality and pop rate 
undesired = ['Admin1', 'SubDiv', 'Sex', 'List', 'Frmat', 'IM_Frmat', 'Cause']
modified_dict = {}
for i in mortality_data_2002.columns.values:
    if i not in undesired:
       modified_dict[i] = []

modified_df = pd.DataFrame(modified_dict)
list_delete_keys = []

for key in country_code_map: 
    temp_mortality_df = mortality_data_2002[mortality_data_2002['Country'] == key] # filter to the country
    if temp_mortality_df.empty:
        list_delete_keys.append(key)
    else:
        for i in range (1980, 2003):
            temp_temp_mortality_df = temp_mortality_df.loc[temp_mortality_df['Year'] == i] # filter to the year 
            if temp_temp_mortality_df.empty:
                continue
            else:
                modified_df.loc[modified_df.shape[0]] = ['None'] * modified_df.shape[1]
                modified_df.loc[modified_df.shape[0]-1, ['Country']] = key
                modified_df.loc[modified_df.shape[0]-1, ['Year']] = i
                column_name = modified_df.columns.values 
                for c in column_name[2:]:
                    if c in temp_temp_mortality_df.columns.values:
                        modified_df.loc[modified_df.shape[0]-1, [c]] = temp_temp_mortality_df[c].sum()
            
                # modified_df.index = modified_df.index + 1
                # modified_df = modified_df.sort_index()      

In [14]:
# delete keys from dictionary
for key in list_delete_keys:
    del country_code_map[key]

In [16]:
# building the mortality model 
country_mortality_model = {}
final_beginning_year = 9999
for key in country_code_map:
    # isolate country
    temp_modified_df = modified_df[modified_df['Country'] == key]
    # temp_modified_df.drop(columns=['Country'], inplace = True) 
    
    # finding alpha 
    last_year = temp_modified_df['Year'].max()
    
    if last_year < final_beginning_year:
        final_beginning_year = last_year
    
    first_year = temp_modified_df['Year'].min()
    duration = last_year - first_year + 1
    last_row = temp_modified_df[temp_modified_df['Year'] == last_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
    first_row = temp_modified_df[temp_modified_df['Year'] == first_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
    aphla = (last_row - first_row) / duration
    
    # finding beta and k 
    m_hat = (last_row - aphla)
    s, u, v = np.linalg.svd(m_hat)
    scalar = s[0][0]
    k = scalar * u[0]
    b = v[0] / scalar 

    country_mortality_model[key] = {'a': aphla, 'd': duration}
    
    
    
        

In [ ]:
final_ending_year = 2050
mortality_pred_all = []
current_year = final_beginning_year
while current_year <= final_ending_year:
    mortality_pred_year = {}
    current_year += 1
    for key in country_code_map: 
        # isolate the data to the target data row \n",
        temp_modified_df = modified_df[(modified_df['Country'] == key)]  
        # check if there is real data for this year for this country\n",
        if current_year <= 2002:
            temp_modified_df = temp_modified_df[modified_df['Year'] == current_year + 1]
            if temp_modified_df.empty:
                temp_modified_df = temp_modified_df.loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
                mortality_pred_year[key] = temp_modified_df 
                continue
 
        # clean the data for the row \n",
        temp_modified_df = temp_modified_df[modified_df['Year'] == current_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
    
        # find the aphla, b and k value for the model for future \n",
        aphla = country_mortality_model[key]['a']
        m_hat = (temp_modified_df - aphla)
        display(m_hat)
        s, u, v = np.linalg.svd(m_hat)
        scalar = s[0][0]
        k = scalar * u[0]
        b = v[0] / scalar 
  
        # compute the next years result \n",
        change = aphla + b * k,
